# Lib

In [ ]:
!pip install openai==1.2.0

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import openai
from openai import OpenAI
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
client = OpenAI(
  api_key="Enter_Your_OpenAI_Key_Here"
)
def get_completion(prompt, model = "gpt-4-turbo-preview", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# DA schema

In [ ]:
rawEx = pd.read_csv('/Data/da_schema.csv').iloc[:25]

In [ ]:
rawEx['combined'] = rawEx.apply(lambda x: list([x['exampleFromPaper'],
                                                x['example1'],
                                                x['example2'],
                                                x['example3']]),axis=1)

In [ ]:
rawEx['combined'] = rawEx['combined'].apply(lambda x:[term for term in x if str(term) != 'nan'])

In [ ]:
role = ['Tutor', 'Tutor & Student', 'Tutor', 'Tutor & Student',
        'Tutor', 'Student','Tutor',
        'Tutor','Tutor & Student','Student',
        'Student','Tutor','Tutor',
        'Tutor','Tutor',
        'Tutor & Student', 'Tutor & Student','Tutor & Student',
        'Tutor & Student', 'Tutor & Student', 'Tutor & Student', 'Tutor & Student',
        'Tutor', 'Student', 'Student',]

In [ ]:
da = rawEx['DA']
sample = rawEx['combined']

In [ ]:
s = pd.DataFrame({"Dialogue Act": da, 'Speakers': role, 'Examples': sample})
sc = s.apply(lambda x: x.to_json(), axis=1).to_json()

# Prompting

In [ ]:
trans_dir = 'Directory for transcript files'
names = os.listdir(trans_dir)

In [ ]:
for n in names:
  data = pd.read_csv(trans_dir+n)
  data['Turn'] = range(len(data))
  d = data[['Turn', 'Speaker', 'Utterance']].iloc[0].to_json()
  prompt = f"""I provided a schema for dialogue act classification in the dictionary, I provide the name of the dialogue act, the speaker,
  and an example of dialogue act in the  dictionary: {sc}. Based on the speaker and the content of the utterance, your task is to select the appropriate dialogue act for the given sentence: {d} Only output the name of the dialogue act for the current sentence"""
  r = get_completion(prompt)
  l = [r]
  for i in range(1,len(data)):
    d = data[['Turn', 'Speaker', 'Utterance']].iloc[i].to_json()
    dp = data[['Turn', 'Speaker', 'Utterance']].iloc[i-1].to_json()
    prompt = f"""I provided a schema for dialogue act classification in the dictionary, I provide the name of the dialogue act, the speaker,
  and an example of dialogue act in the  dictionary: {sc}. Based on the speaker and the content of the utterance, your task is to select the appropriate dialogue act for the given sentence: {d},
  given that the previous the previous sentence is {dp} Only output the name of the dialogue act for the current sentence"""
    r = get_completion(prompt)
    l.append(r)
  data['DA']=l
  data.to_csv('Directory for DA labeled transcript files'+n)
